## Bibliotecas

In [338]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from datetime import datetime, timedelta
from sklearn.svm import SVC  
from sklearn.neighbors import KNeighborsClassifier


# Coleta de Dados

In [339]:
df_inicial = pd.read_excel("base_completa_sem_filtro.xlsx", sheet_name="completo")

# Pré-processamento de Dados

Análise do tipo de dado presente

In [340]:
df_inicial.describe()

,Data de Embarque,Data PCP,Data de Entrega,Data Acordada,Data do Faturamento,Situação da Entrega,Situação do Pedido,LINHA,Pedido Terceiro,Terceiro Centralizador,...,Unidade,Peso Líquido Estimado,Peso Bruto Estimado,Volume Estimado,Peso Embarcado,Volume Embarcado,Limite Mínimo Embarque,Limite Máximo Embarque,Data de Cancelamento,Motivo de Cancelamento
count,37335,37054,37315,37315,28259,37315,37315,37315,14535,37315,...,37315,37315,37315,37315.000,27323,27323,36971,37314,1,1
unique,242,228,265,280,172,3,13,18,2046,203,...,4,713,717,756.000,5026,359,8,40,1,1
top,2023-04-27 00:00:00,2023-07-06 00:00:00,2023-04-14 00:00:00,2023-03-17 00:00:00,2023-04-28 00:00:00,Faturada,Faturado,CAJAMAR MOÍDA ATM,18304526,1042,...,CXS,15,15,1.734,30,1,0,3,Data de Cancelamento,Motivo de Cancelamento
freq,361,358,622,583,328,28258,28258,19518,177,9020,...,24727,4922,4922,4156.000,1678,6214,33815,18908,1,1


In [341]:
df_inicial.dtypes

Data de Embarque                  object
Data PCP                          object
Data de Entrega                   object
Data Acordada                     object
Data do Faturamento               object
Situação da Entrega               object
Situação do Pedido                object
LINHA                             object
Pedido Terceiro                   object
Terceiro Centralizador            object
Nome do Terceiro Centralizador    object
Código Terceiro                   object
Nome do Terceiro                  object
Cidade                            object
UF                                object
Pedido                            object
Data Emissão Pedido               object
Hora Emissão Pedido               object
Usuário Digitador                 object
Usuário Responsável               object
Código Instrução Filial           object
Data da Instrução                 object
Data Liberação Pedido             object
Placa do Veículo                  object
Produto         

In [342]:
df_inicial.columns

Index(['Data de Embarque', 'Data PCP', 'Data de Entrega', 'Data Acordada',
       'Data do Faturamento', 'Situação da Entrega', 'Situação do Pedido',
       'LINHA', 'Pedido Terceiro', 'Terceiro Centralizador',
       'Nome do Terceiro Centralizador', 'Código Terceiro', 'Nome do Terceiro',
       'Cidade', 'UF', 'Pedido', 'Data Emissão Pedido', 'Hora Emissão Pedido',
       'Usuário Digitador', 'Usuário Responsável', 'Código Instrução Filial',
       'Data da Instrução', 'Data Liberação Pedido', 'Placa do Veículo',
       'Produto', 'Nome Produto', 'Qtde', 'Unidade', 'Peso Líquido Estimado',
       'Peso Bruto Estimado', 'Volume Estimado', 'Peso Embarcado',
       'Volume Embarcado', 'Limite Mínimo Embarque', 'Limite Máximo Embarque',
       'Data de Cancelamento', 'Motivo de Cancelamento'],
      dtype='object')

In [343]:
df_inicial.shape

(37335, 37)

In [344]:
df_inicial.isnull()

,Data de Embarque,Data PCP,Data de Entrega,Data Acordada,Data do Faturamento,Situação da Entrega,Situação do Pedido,LINHA,Pedido Terceiro,Terceiro Centralizador,...,Unidade,Peso Líquido Estimado,Peso Bruto Estimado,Volume Estimado,Peso Embarcado,Volume Embarcado,Limite Mínimo Embarque,Limite Máximo Embarque,Data de Cancelamento,Motivo de Cancelamento
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,True,False,False,True,False,False,False,True,False,...,False,False,False,False,True,True,False,False,True,True
2,False,True,False,False,True,False,False,False,True,False,...,False,False,False,False,True,True,False,False,True,True
3,False,True,False,False,True,False,False,False,True,False,...,False,False,False,False,True,True,False,False,True,True
4,False,True,False,False,True,False,False,False,True,False,...,False,False,False,False,True,True,False,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37330,False,False,False,False,True,False,False,False,True,False,...,False,False,False,False,True,True,False,False,True,True
37331,False,False,False,False,True,False,False,False,True,False,...,False,False,False,False,True,True,False,False,True,True
37332,False,False,False,False,True,False,False,False,True,False,...,False,False,False,False,True,True,False,False,True,True
37333,False,False,False,False,True,False,False,False,True,False,...,False,False,False,False,True,True,False,False,True,True


In [345]:
df_inicial.value_counts
df_inicial['Produto']

0        Produto
1         370321
2         379395
3         378979
4         378980
          ...   
37330     375622
37331     375621
37332     375635
37333     375622
37334     375621
Name: Produto, Length: 37335, dtype: object

Limpeza dos dados

In [346]:
df_intermediario = df_inicial.copy()
df_intermediario = df_intermediario.drop_duplicates() # Retirando dados duplicados
df_intermediario = df_intermediario.drop(0) # Retirando primeira coluna pois veio com o titulo da coluna
df_intermediario = df_intermediario.drop(df_intermediario[df_intermediario['Data PCP'] != df_intermediario['Data de Embarque']].index) # Retirando linhas onde data PCP e data de embarque não são iguais
df_intermediario = df_intermediario.drop(df_intermediario[df_intermediario['Situação da Entrega'] != 'Faturada'].index) # Retirando pedidos sem faturamento de entrega
colunas_desejadas = ['Data PCP', 'LINHA', 'Código Terceiro', 'Cidade', 'UF', 'Peso Líquido Estimado', 'Terceiro Centralizador', 'Produto', 'Data Acordada']
df_intermediario = df_intermediario[colunas_desejadas]

Transformação de Dados

In [347]:
# Puxar o dia da semana
df_intermediario['Data Acordada'] = pd.to_datetime(df_intermediario['Data Acordada'], format='%d/%m/%Y') # Passar coluna Data PCP de string para tipo date
df_intermediario['numero_dia_acordada'] = df_intermediario['Data Acordada'].dt.dayofweek # Obter número do dia da semana, onde 0 é segunda-feira e 6 é domingo
df_intermediario.dtypes


Data PCP                          object
LINHA                             object
Código Terceiro                   object
Cidade                            object
UF                                object
Peso Líquido Estimado             object
Terceiro Centralizador            object
Produto                           object
Data Acordada             datetime64[ns]
numero_dia_acordada                int32
dtype: object

In [348]:
# Puxar o dia da semana
df_intermediario['Data PCP'] = pd.to_datetime(df_intermediario['Data PCP'], format='%d/%m/%Y') # Passar coluna Data PCP de string para tipo date
df_intermediario['numero_dia_pcp'] = df_intermediario['Data PCP'].dt.dayofweek # Obter número do dia da semana, onde 0 é segunda-feira e 6 é domingo
df_intermediario.dtypes

Data PCP                  datetime64[ns]
LINHA                             object
Código Terceiro                   object
Cidade                            object
UF                                object
Peso Líquido Estimado             object
Terceiro Centralizador            object
Produto                           object
Data Acordada             datetime64[ns]
numero_dia_acordada                int32
numero_dia_pcp                     int32
dtype: object

In [349]:
# Puxar a faixa de peso no carregamento
limites_peso = [0, 500, 1500, 5000, float('inf')]
rotulos_peso = [500, 1500, 5000, 28000]
df_intermediario['faixa_de_peso'] = pd.cut(df_intermediario['Peso Líquido Estimado'], bins=limites_peso, labels=rotulos_peso)
df_intermediario['faixa_de_peso'] = df_intermediario['faixa_de_peso'].astype(int)
df_intermediario.dtypes

Data PCP                  datetime64[ns]
LINHA                             object
Código Terceiro                   object
Cidade                            object
UF                                object
Peso Líquido Estimado             object
Terceiro Centralizador            object
Produto                           object
Data Acordada             datetime64[ns]
numero_dia_acordada                int32
numero_dia_pcp                     int32
faixa_de_peso                      int32
dtype: object

In [350]:
# Passando código de terceiro de str para int
df_intermediario['Código Terceiro'] = df_intermediario['Código Terceiro'].astype(int)
df_intermediario['Terceiro Centralizador'] = df_intermediario['Terceiro Centralizador'].astype(int)
df_intermediario.dtypes

Data PCP                  datetime64[ns]
LINHA                             object
Código Terceiro                    int32
Cidade                            object
UF                                object
Peso Líquido Estimado             object
Terceiro Centralizador             int32
Produto                           object
Data Acordada             datetime64[ns]
numero_dia_acordada                int32
numero_dia_pcp                     int32
faixa_de_peso                      int32
dtype: object

In [351]:
# Adicionando encoders nas colunas LINHA, Cidade e UF
label_encoder = LabelEncoder()
df_intermediario['Linha_encoded'] = label_encoder.fit_transform(df_intermediario['LINHA'])
df_intermediario['Cidade_encoded'] = label_encoder.fit_transform(df_intermediario['Cidade'])
df_intermediario['Uf_encoded'] = label_encoder.fit_transform(df_intermediario['UF'])
df_intermediario.dropna()

,Data PCP,LINHA,Código Terceiro,Cidade,UF,Peso Líquido Estimado,Terceiro Centralizador,Produto,Data Acordada,numero_dia_acordada,numero_dia_pcp,faixa_de_peso,Linha_encoded,Cidade_encoded,Uf_encoded
6,2023-08-27,CAJAMAR MOÍDA ATM,2684242,FRANCO DA ROCHA,SP,90,1416307,378979,2023-09-01,4,6,500,3,35,8
7,2023-08-27,CAJAMAR MOÍDA ATM,2684242,FRANCO DA ROCHA,SP,36,1416307,378980,2023-09-01,4,6,500,3,35,8
8,2023-08-27,CAJAMAR TUBETE,2684242,FRANCO DA ROCHA,SP,566.01,1416307,370321,2023-09-01,4,6,1500,4,35,8
9,2023-08-27,CAJAMAR MOÍDA ATM,3763760,GUARULHOS,SP,54,1416307,378979,2023-09-01,4,6,500,3,40,8
10,2023-08-27,CAJAMAR MOÍDA ATM,3763760,GUARULHOS,SP,18,1416307,378980,2023-09-01,4,6,500,3,40,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37102,2023-09-13,CAJAMAR MOÍDA ATM,28253,TAUBATE,SP,18,5135,378980,2023-09-15,4,2,500,3,134,8
37103,2023-09-13,CAJAMAR TUBETE,28253,TAUBATE,SP,37.734,5135,370321,2023-09-15,4,2,500,4,134,8
37104,2023-09-13,CAJAMAR MOÍDA ATM,2247448,SAO JOSE DOS CAMPOS,SP,66,2064982,378979,2023-09-15,4,2,500,3,121,8
37105,2023-09-13,CAJAMAR MOÍDA ATM,2247448,SAO JOSE DOS CAMPOS,SP,78,2064982,378980,2023-09-15,4,2,500,3,121,8


# Divisão de Dados

In [370]:
data_mais_recente = df_intermediario['Data PCP'].max()

data_inicio = data_mais_recente - timedelta(days=40)  


df_recortado = df_intermediario.loc[(df_intermediario['Data PCP'] >= data_inicio) & (df_intermediario['Data PCP'] <= data_mais_recente)]

df_recortado


,Data PCP,LINHA,Código Terceiro,Cidade,UF,Peso Líquido Estimado,Terceiro Centralizador,Produto,Data Acordada,numero_dia_acordada,numero_dia_pcp,faixa_de_peso,Linha_encoded,Cidade_encoded,Uf_encoded
6,2023-08-27,CAJAMAR MOÍDA ATM,2684242,FRANCO DA ROCHA,SP,90,1416307,378979,2023-09-01,4,6,500,3,35,8
7,2023-08-27,CAJAMAR MOÍDA ATM,2684242,FRANCO DA ROCHA,SP,36,1416307,378980,2023-09-01,4,6,500,3,35,8
8,2023-08-27,CAJAMAR TUBETE,2684242,FRANCO DA ROCHA,SP,566.01,1416307,370321,2023-09-01,4,6,1500,4,35,8
9,2023-08-27,CAJAMAR MOÍDA ATM,3763760,GUARULHOS,SP,54,1416307,378979,2023-09-01,4,6,500,3,40,8
10,2023-08-27,CAJAMAR MOÍDA ATM,3763760,GUARULHOS,SP,18,1416307,378980,2023-09-01,4,6,500,3,40,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37102,2023-09-13,CAJAMAR MOÍDA ATM,28253,TAUBATE,SP,18,5135,378980,2023-09-15,4,2,500,3,134,8
37103,2023-09-13,CAJAMAR TUBETE,28253,TAUBATE,SP,37.734,5135,370321,2023-09-15,4,2,500,4,134,8
37104,2023-09-13,CAJAMAR MOÍDA ATM,2247448,SAO JOSE DOS CAMPOS,SP,66,2064982,378979,2023-09-15,4,2,500,3,121,8
37105,2023-09-13,CAJAMAR MOÍDA ATM,2247448,SAO JOSE DOS CAMPOS,SP,78,2064982,378980,2023-09-15,4,2,500,3,121,8


In [369]:
colunas_features = ['Código Terceiro', 'Linha_encoded', 'Cidade_encoded', 'Uf_encoded', 'numero_dia_acordada', 'Terceiro Centralizador']
X = df_recortado[colunas_features]
y = df_recortado['numero_dia_pcp']

# Seleção de Algoritmo

In [373]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

modelos = {
    'Random Forest': RandomForestClassifier(),
    'Decision Tree': DecisionTreeClassifier(),
    'SVM': SVC(),
    'KNN': KNeighborsClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    'AdaBoost': AdaBoostClassifier(),
}

resultados = {}

for nome_modelo, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    acuracia = accuracy_score(y_test, y_pred)
    resultados[nome_modelo] = acuracia

for nome_modelo, acuracia in resultados.items():
    print(f'Acurácia do modelo {nome_modelo}: {acuracia * 100:.2f}%')

Acurácia do modelo Random Forest: 81.53%
Acurácia do modelo Decision Tree: 80.23%
Acurácia do modelo SVM: 43.69%
Acurácia do modelo KNN: 74.77%
Acurácia do modelo Gradient Boosting: 80.36%
Acurácia do modelo AdaBoost: 53.58%


In [355]:
X

,Código Terceiro,Linha_encoded,Cidade_encoded,Uf_encoded,numero_dia_acordada
6,2684242,3,35,8,4
7,2684242,3,35,8,4
8,2684242,4,35,8,4
9,3763760,3,40,8,4
10,3763760,3,40,8,4
...,...,...,...,...,...
37102,28253,3,134,8,4
37103,28253,4,134,8,4
37104,2247448,3,121,8,4
37105,2247448,3,121,8,4


# Treinamento do Modelo

# Validação e Ajuste de Hiperparâmetros

# Avaliação do Modelo

# Interpretação do Modelo

# Implantação do Modelo

# Monitoramento e Manutenção

# Documentação

# Treinamento e Reciclagem